# 05-03. 트리의 앙상블

## 정형 데이터와 비정형 데이터
- 정형 데이터(structured data): 특정한 구조로 구성된 데이터로 CSV나 데이터베이스, 엑셀에 저장되기 용이한 데이터
 - 온라인 쇼핑몰의 상품, 구매한 쇼핑 정보 등 프로그래머가 다루는 데부분의 데이터가 정형 데이터에 속함
 - 지금까지 배운 머신러닝 알고리즘은 정형 데이터에 잘 맞는 알고리즘
- 비정형 데이터(unstructured data): 텍스트, 사진, 음악 등 데이터베이스나 엑셀에 표현하기 어려운 데이터
 - 비정형 데이터에서는 신경망 알고리즘을 사용함

## 앙상블 학습(ensemble learning)
- 여러 개의 개별 모델을 조합하여 최적의 모델로 일반화하는 방법
- 정형 데이터를 다루는 알고리즘 중 가장 뛰어난 성과를 보임

### 랜덤 포레스트(Random Forest)
- 가장 대표적인 앙상블 학습 알고리즘
- 결정 트리를 랜덤하게 생성하고 각 결정 트리의 예측을 사용해 최종 예측을 계산
- 부트스트랩 샘플(bootstrap sample): 각 트리를 훈련하기 위해 종복을 허용하여 랜덤으로 샘플링한 데이터이며, 일반적으로 훈련 세트의 크기과 동일하게 생성
- 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 선택한 후 그 중에서 최선의 분할을 탐색함
- 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 방지하며 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음
- RandomForestClassifier: 사이킷런의 분류 클래스
 - 기본적으로 100개의 결정 트리를 사용
 - 무작위로 특성을 선택할 때 전체 특성 개수의 제곱근만큼의 특성을 선택
 - DecisionTreeClassifier가 제공하는 주요 매개변수를 모두 제공
 - OOB(out of bag) 샘플: 부트스트랩 샘플에 포함되지 않고 남아 있는 샘플이며 이 샘플을 검증 샘플처럼 이용해 부트스트랩 샘플로 훈련한 트리를 평가할 수 있음
- RandomForestRegressor: 사이킷런의 회귀 클래스
 - 특성을 선택하지 않고 전체 특성 사용

In [ ]:
# 데이터를 불러온 후 세트 분할

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine= pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target= wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# RandomForestClassifier를 cross_validate()로 교차 검증

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
# 특성의 중요도 출력

rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 앞서 결정 트리에서 구했던 중요도[0.12345626, 0.86862934, 0.0079144]에 비해 2번째 특성(당도)의 중요도가 감소
- 랜덤 포레스트는 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에, 하나의 특성에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여할 기회를 얻음

In [ ]:
# OOB 샘플로 훈련을 평가

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)  # OOB 점수를 얻기 위해서는 oob_score를 True로 바꾸어야 함
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


- 교차 검증에서 얻은 것과 비슷한 결과 출력   
→ OOB 점수를 교차 검증 대신 사용하면 더 많은 샘플을 훈련 세트에 사용 가능

### 엑스트라 트리(Extra Tree)
- 랜덤 포레스트와 매우 비슷하게 동작하며 대부분의 매개변수를 지원
- 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트를 사용하며, 노드를 분할할 때 무작위로 분할(splitter='random'인 결정 트리를 사용)
- 많은 트리를 앙상블하기 때문에 과대적합을 방지하고 검증 세트의 점수를 향상시킴
- 랜덤 포레스트에 비해 무작위성이 크기 때문에 더 많은 결정 트리를 훈련해야하지만 랜덤으로 노드를 분할하기 때문에 더 빠른 계산이 가능
- ExtraTreesClassifier: 사이킷런의 분류 클래스
- ExtraTreesRegressor: 사이킷런의 회귀 클래스

In [ ]:
# ExtraTreesClassifier 사용

from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


- 예제의 특성이 많지 않아 랜덤 포레스트와 비슷한 결과가 나왔으나 일반적으로 엑스트라 트리가 무작위성이 크기 때문에 더 많은 결정 트리를 훈련해야함

In [ ]:
# 특성 중요도 출력

et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### 그레이디언트 부스팅(gradient boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- 경사 하강법을 사용해 순차적으로 오차를 반영하는 새로운 트리를 만들어 나감
 - 분류에서는 로지스틱 손실 함수를, 회귀에서는 평균 제곱 오차 함수를 사용
- 낮은 지점으로 천천히 조금씩 이동해야 하기 때문에 깊이가 얕은 트리를 사용하며 학습률 매개변수(default: 0.1)로 속도를 조절
- 결정 트리의 개수를 늘려도 잘 과대적합되지 않으며 학습률을 증가시키고 트리의 개수를 늘리면 더 향상될 수도 있음
- 랜덤 포레스트에 비해 조금 더 높은 성능을 보이나 순서대로 트리를 추가하기 때문에 훈련 속도가 느림
- GradientBoostingClassifier: 사이킷런의 분류 클래스
 - 기본적으로 깊이가 3인 결정 트리 100개를 사용
- GradientBoostingRegressor: 사이킷런의 회귀 클래스

In [ ]:
# GradientBosstingClassifier를 교차검증

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
# 트리의 개수와 학습률을 증가

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


- 트리의 개수를 5배로 늘렸지만 과대적합을 잘 억제하고 있음

In [ ]:
# 특성 중요도 출력

gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


- 그레이디언트 부스팅이 랜덤 포레스트에 비해 일부 특성(당도)에 더 집중

In [ ]:
gb = GradientBoostingClassifier(subsample=0.5, random_state=42)  # subsample을 조정해 훈련 세트의 비율을 조절할 수 있음(default: 1.0)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8870982526503335 0.8712741541422966


### 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
- 입력 데이터를 히스토그램으로 변환하고 256개의 구간으로 나누어 각 구간에 속하는 데이터의 개수를 기록하여 사용하는 그레이디언트 부스팅 기법
- 노드 분할 시 최적의 분항르 매우 빠르게 찾을 수 있음
- 256개의 구간 중 하나를 떼어놓고 누락된 값을 위해 사용하기 때문에 누락된 특성에 대한 전처리가 필요하지 않음
- HistGradientBoostingClassifier: 사이킷런의 분류 클래스
 - 트리의 개수를 지정하는 estimators 대신 부스팅 반복 횟수를 지정하는 max_iter 사용
- HistGradientBoostingRegressor: 사이킷런의 회귀 클래스

In [ ]:
# from sklearn.experimental import enable_hist_gradient_boosting 버전 1.0부터 HistGradientBoostingClassifier만 import 해도 됨
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


- 과대적합을 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능 제공

 - permutation_importance(): 특성을 하나씩 랜덤하게 섞고 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산하는 함수   
 사이킷런이 제공하는 추정기 모델에 모두 사용 가능
  - 함수가 반환하는 객체는 특성 중요도(importances), 중요도의 평균(importances_mean), 중요도의 표준 편차(importances_std)를 담고 있음
  - n_repeats: 랜덤하게 섞을 횟수(default: 5)

In [ ]:
# permutation_importance()로 훈련 세트에서의 특성 중요도 계산

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
# 테스트에서의 특성 중요도 계산

hgb.fit(test_input, test_target)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.14092308 0.27415385 0.13746154]


- 그레이디언트 부스팅과 비슷하게 당도에 더 집중하고 있음을 확인할 수 있음

In [ ]:
# 테스트 세트에서의 성능 최종 확인
hgb.score(test_input, test_target)

0.9730769230769231

#### 히스토그램 기반 그레이디언트 부스팅을 구현한 다른 라이브러리
- XGBoost
 - 사이킷런의 cross_validate() 함수와 함께 사용 가능
- LightGBM
 - 마이크로소프트에서 제작한 라이브러리
 - 최신 기술을 많이 적용하고 있어 인기가 높아지고 있음

In [ ]:
# XGBoost

from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [ ]:
# LightGBM

from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
